# 1. 
## 필요한 framework, GPU 및 초기 parameter 세팅


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# 사용가능한 GPU 할당
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")

learning_rate = 0.001 
batch_size = 100  
num_classes = 10  
epochs = 5

# 2.
## torchvision에서 제공하는 MNIST 데이터 로드

In [ ]:
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,             # train data
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,            # test data
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# 3.
## Model을 class로 만들기

In [ ]:
 class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # num of input_channels = 1, num of out_channels = 10, kernel size = 5, stribe = 1, padding = 0 인 convolution layer
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # input_channels = 10, out_channels = 20, kernel size = 5, stribe = 1, padding = 0 인 convolution layer
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # 0.25의 확률로 각 노드를 0으로 turn-off
        self.mp = nn.MaxPool2d(2) # kernel_size=2로 max값으로 pooling
        self.fc1 = nn.Linear(320,100)  # input_size = 320, output_size = 100인 Fully-connected layer
        self.fc2 = nn.Linear(100,10)  # input_size = 100, output_size = 10인 Fully-connected layer

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # convolution layer(in:1, out:10) 연산 후 maxpooling 후 relu activation 함수 적용
        x = F.relu(self.mp(self.conv2(x))) # convolution layer(in:10, out:20) 연산 후 maxpooling 후 relu activation 함수 적용
        x = self.drop2D(x)  # 0.25 확률로 dropout
        x = x.view(x.size(0), -1) # Flatten 시킴
        x = self.fc1(x)  # Fully connected linear layer 1번 진행 320->100
        x = self.fc2(x)  # Linear 한번 더 진행 100->10
        return F.log_softmax(x)  # softmax함수의 log값 반환
 


# 4.
## 모델, 평가기준(loss 함수), optim 함수 정의

In [ ]:
model = ConvNet().to(device)  # instance 생성
criterion = nn.CrossEntropyLoss().to(device)    # loss 함수로 CrossEntropyLoss 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)  # weight parameter 학습 시 Adam 알고리즘 사용;
 

# 5.
## training

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)  # data를 GPU(또는 cpu)에 얹음
        target = target.to(device)  
        optimizer.zero_grad() # 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : 새로운 가중치 편향에 대한 새로운 기울기를 구하기 위해
        hypothesis = model(data) # predicte y값 계산
        cost = criterion(hypothesis, target) # loss 계산
        cost.backward() # loss의 gradient 계산
        optimizer.step() # parameter update
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 6.
## Test

In [ ]:
model.eval()    # test mode로 설정
with torch.no_grad():   # test시에는 gradient 추적하지 않음
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)  
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() # 맞은 개수 계산
        
    print('Test Accuracy: ', 100.*correct/total, '%')  # 정확도 계산 = 맞은 개수 / 전체 개수 * 100